In [ ]:
import itertools
import networkx as nx
import numpy as np

import network

from preprocess_data import *
from datetime import date
from enum import Enum

import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

from scipy import stats

In [ ]:
def regular_networks():
    for game, venue, (situation, pp) in itertools.product(
        GAMES,
        [Venue.HOME, Venue.AWAY],
        [(Situation.REGULAR, None)]
    ):
        res = build_networks(Game(game), venue=venue, situation=situation, pp=pp)
        if res is not None:
            nw = network.process_graph_for_analysis(res["position_pass_network"], make_undirected=False)
            degrees = dict(nw.degree(weight=Weight.N_PASSES))
            dm = network.degree_mean(nw)
            dstd = network.degree_std(nw)
            time = calculate_time(Game(game), None)
            yield {"degree_mean": degree_mean, "degree_std": degree_std, "time": time}

In [ ]:
def power_play_networks():
    for game, venue, pp in itertools.product(
        GAMES,
        [Venue.HOME, Venue.AWAY],
        list(map(PowerPlay, range(1, MAX_PENALTY_NUMBER + 1)))
    ):
        res = build_networks(Game(game), venue=venue, situation=Situation.POWER_PLAY, pp=pp)
        if res is not None:
            nw = network.process_graph_for_analysis(res["position_pass_network"], make_undirected=False)
            if len(nw.nodes) == 5:
                degree_man = network.degree_mean(nw)
                degree_std = network.degree_std(nw)
                time = calculate_time(Game(game), [pp])
                yield {"degree_mean": dm, "degree_std": degree_std, "time": time}

In [ ]:
# (Weighted) Degree Mean: Regular vs Power Play
fig, axs = plt.subplots(2, 1, sharex=True, sharey=True)
fig.set_figheight(10)
fig.set_figwidth(15)

x_reg = [stats["degree_mean"] / stats["time"] for stats in regular_networks()]
x_pp  = [stats["degree_mean"] / stats["time"] for stats in power_play_networks()]
n, bins, patches = axs[0].hist(x_reg, facecolor='blue', alpha=0.5)
n, bins, patches = axs[1].hist(x_pp, facecolor='blue', alpha=0.5)
axs[0].set_title("Regular Networks")
axs[1].set_title("Power Play Networks")

plt.show()

stats.ttest_ind(x_reg, x_pp, alternative="less")

In [ ]:
# (Weighted) Degree Std: Regular vs Power Play
fig, axs = plt.subplots(2, 1, sharex=True, sharey=True)
fig.set_figheight(10)
fig.set_figwidth(15)

x_reg = [stats["degree_std"] / stats["time"] for stats in regular_networks()]
x_pp  = [stats["degree_std"] / stats["time"] for stats in power_play_networks()]
n, bins, patches = axs[0].hist(x_reg, facecolor='blue', alpha=0.5)
n, bins, patches = axs[1].hist(x_pp, facecolor='blue', alpha=0.5)
axs[0].set_title("Regular Networks")
axs[1].set_title("Power Play Networks")

plt.show()

stats.ttest_ind(x_reg, x_pp, alternative="less")